In [6]:
from tensorflow import keras
from tensorflow.keras.optimizers import Adam,SGD,RMSprop,Adadelta
from tensorflow.keras.layers import Dense, Activation, Flatten
from tensorflow.keras.callbacks import ModelCheckpoint,ReduceLROnPlateau
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score,mean_squared_error
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import time

In [7]:
main_path_energy ='../data_base/paths_energy/'
main_path_total_energy = '../data_base/paths_totals/'

energy_path11_pd = pd.read_csv(main_path_energy + 'feat_pathv11_TM.csv')
energy_path12_pd = pd.read_csv(main_path_energy + 'feat_pathv12_TM.csv')
energy_path13_pd = pd.read_csv(main_path_energy + 'feat_pathv13_TM.csv')

energy_path_list = [energy_path11_pd,energy_path12_pd,energy_path13_pd]
complete_energy_pd = pd.concat(energy_path_list)

total_path11_pd = pd.read_csv(main_path_total_energy + 'totalF_pathv11.csv')
total_path12_pd = pd.read_csv(main_path_total_energy + 'totalF_pathv12.csv')
total_path13_pd = pd.read_csv(main_path_total_energy + 'totalF_pathv13.csv')

total_path_list = [total_path11_pd,total_path12_pd,total_path13_pd]
complete_total_pd = pd.concat(total_path_list)

In [8]:
eps = 5
complete_energy_pd = complete_energy_pd[(complete_energy_pd['sim_point_dist']-complete_energy_pd['teo_point_dist']) < eps]
print(f'new shape of complete_energy_pd = {complete_energy_pd.shape}')

new shape of complete_energy_pd = (3957, 17)


In [9]:
X = complete_energy_pd.drop(columns=["missing_points","path_num","sim_drone_time","Energy","sim_point_dist","sim_Xdist","sim_Ydist","avg_error_target_dist","avg_energy"],axis=1)
y = complete_energy_pd["Energy"].to_numpy()

X = X.to_numpy()

x_train,x_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=35)

scaler2 = StandardScaler()

x_train = scaler2.fit_transform(x_train)
x_test = scaler2.transform(x_test)

print(f'Train size/Test size {len(x_train),len(x_test)}')

Train size/Test size (3165, 792)


#### Pruebas con Optimizer: Adam,variando lr

In [10]:
start_time = time.time()
NN_model = Sequential()

NN_model.add(Dense(128,kernel_initializer='normal',input_dim=8,activation='relu'))

NN_model.add(Dense(128,kernel_initializer='normal',activation='relu'))
NN_model.add(Dense(256,kernel_initializer='normal',activation='relu'))
NN_model.add(Dense(256,kernel_initializer='normal',activation='relu'))
NN_model.add(Dense(128,kernel_initializer='normal',activation='relu'))

NN_model.add(Dense(1, kernel_initializer='normal',activation='linear'))

lr = [0.1,0.01,0.001,0.0001,0.00001]
results_Adam = np.empty((4,5),dtype=object)

for idx,i in enumerate(lr):
    NN_model.compile(loss="mean_squared_logarithmic_error", optimizer=Adam(learning_rate=i), metrics=["mean_squared_logarithmic_error"])
    model = NN_model.fit(x_train, y_train, epochs=250, batch_size=32, validation_split = 0.2)
    
    y_predict = NN_model.predict(x_test)
    r2 = r2_score(y_test,y_predict)
    RMSE = mean_squared_error(y_test,y_predict,squared=False)
    MSE = mean_squared_error(y_test,y_predict)
    
    results_Adam[0,idx] = r2
    results_Adam[1,idx] = RMSE
    results_Adam[2,idx] = MSE
    results_Adam[3,idx] = model

print(results_Adam)
print('Execution time = ' + str(time.time() - start_time))

Train on 2532 samples, validate on 633 samples
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Epoch 1/250
2532/2532 [==============================] - 0s 131us/sample - loss: 4.5667 - mean_squared_logarithmic_error: 4.5667 - val_loss: 0.1242 - val_mean_squared_logarithmic_error: 0.1242
Epoch 2/250
2532/2532 [==============================] - 0s 67us/sample - loss: 0.1278 - mean_squared_logarithmic_error: 0.1278 - val_loss: 0.1196 - val_mean_squared_logarithmic_error: 0.1196
Epoch 3/250
2532/2532 [==============================] - 0s 67us/sample - loss: 0.1238 - mean_squared_logarithmic_error: 0.1238 - val_loss: 0.1144 - val_mean_squared_logarithmic_error: 0.1144
Epoch 4/250
2532/2532 [==============================] - 0s 67us/sample - loss: 0.1232 - mean_squared_logarithmic_error: 0.1232 - val_loss: 0.1335 - val_mean_squared_logarithmic_error: 0.1335
Epoch 5/250
2532/2532 [==============================] - 0s 66us/sample - loss: 0.1284 - m

In [11]:
print(results_Adam)

[[0.6572490499980612 0.6527524400278993 0.6508330516621172
  0.6506825074927765 0.6506297371944765]
 [884306.7146124276 890088.5009617854 892545.064245821 892737.4549859014
  892804.8839634893]
 [781998365508.6256 792257539544.3983 796636691709.5767 796980163534.7042
  797100560829.0597]


#### SGD with momentum, variando el lr y dejando el momemtum en 0.9

In [12]:
start_time = time.time()
NN_model = Sequential()

NN_model.add(Dense(128,kernel_initializer='normal',input_dim=8,activation='relu'))

NN_model.add(Dense(128,kernel_initializer='normal',activation='relu'))
NN_model.add(Dense(256,kernel_initializer='normal',activation='relu'))
NN_model.add(Dense(256,kernel_initializer='normal',activation='relu'))
NN_model.add(Dense(128,kernel_initializer='normal',activation='relu'))

NN_model.add(Dense(1, kernel_initializer='normal',activation='linear'))

lr = [0.1,0.01,0.001,0.0001,0.00001]
results_SGD = np.empty((4,5),dtype=object)

for idx,i in enumerate(lr):
    NN_model.compile(loss="mean_squared_logarithmic_error", optimizer=SGD(learning_rate=i,momentum=0.9), metrics=["mean_squared_logarithmic_error"])
    model = NN_model.fit(x_train, y_train, epochs=250, batch_size=32, validation_split = 0.2,verbose=0)
    
    y_predict = NN_model.predict(x_test)
    r2 = r2_score(y_test,y_predict)
    RMSE = mean_squared_error(y_test,y_predict,squared=False)
    MSE = mean_squared_error(y_test,y_predict)
    
    results_SGD[0,idx] = r2
    results_SGD[1,idx] = RMSE
    results_SGD[2,idx] = MSE
    results_SGD[3,idx] = model
print('Execution time = ' + str(time.time() - start_time))

Execution time = 256.2498052120209


In [13]:
print(results_SGD)

[[0.6529187511298171 0.6522290842930181 0.6519354482977058
  0.6519215143574366 0.6519335561233899]
 [889875.3254851048 890758.9988814275 891134.9701109249 891152.807171774
  891137.3923346222]
 [791878094907.2213 793451594088.2428 794121534954.599 794153325730.1329
  794125852016.9504]


#### RMSprop variando el learning rate, momentum 0.9

In [14]:
start_time = time.time()
NN_model = Sequential()

NN_model.add(Dense(128,kernel_initializer='normal',input_dim=8,activation='relu'))

NN_model.add(Dense(128,kernel_initializer='normal',activation='relu'))
NN_model.add(Dense(256,kernel_initializer='normal',activation='relu'))
NN_model.add(Dense(256,kernel_initializer='normal',activation='relu'))
NN_model.add(Dense(128,kernel_initializer='normal',activation='relu'))

NN_model.add(Dense(1, kernel_initializer='normal',activation='linear'))

lr = [0.1,0.01,0.001,0.0001,0.00001]
results_RMSprop = np.empty((4,5),dtype=object)

for idx,i in enumerate(lr):
    NN_model.compile(loss="mean_squared_logarithmic_error", optimizer=RMSprop(learning_rate=i,momentum=0.9), metrics=["mean_squared_logarithmic_error"])
    model = NN_model.fit(x_train, y_train, epochs=250, batch_size=32, validation_split = 0.2,verbose=0)
    
    y_predict = NN_model.predict(x_test)
    r2 = r2_score(y_test,y_predict)
    RMSE = mean_squared_error(y_test,y_predict,squared=False)
    MSE = mean_squared_error(y_test,y_predict)
    
    results_RMSprop[0,idx] = r2
    results_RMSprop[1,idx] = RMSE
    results_RMSprop[2,idx] = MSE
    results_RMSprop[3,idx] = model

print('Execution time = ' + str(time.time() - start_time))

Execution time = 236.11199426651


In [15]:
print(results_RMSprop)

[[-194231243.9250411 -194231243.9250411 -194231243.9250411
  -194231243.9250411 -194231243.9250411]
 [21051019616.85083 21051019616.85083 21051019616.85083 21051019616.85083
  21051019616.85083]
 [4.431454269090385e+20 4.431454269090385e+20 4.431454269090385e+20
  4.431454269090385e+20 4.431454269090385e+20]


#### Adadelta, variando el lr

In [16]:
start_time = time.time()
NN_model = Sequential()

NN_model.add(Dense(128,kernel_initializer='normal',input_dim=8,activation='relu'))

NN_model.add(Dense(128,kernel_initializer='normal',activation='relu'))
NN_model.add(Dense(256,kernel_initializer='normal',activation='relu'))
NN_model.add(Dense(256,kernel_initializer='normal',activation='relu'))
NN_model.add(Dense(128,kernel_initializer='normal',activation='relu'))

NN_model.add(Dense(1, kernel_initializer='normal',activation='linear'))

lr = [0.1,0.01,0.001,0.0001,0.00001]
results_Adadel = np.empty((4,5),dtype=object)

for idx,i in enumerate(lr):
    NN_model.compile(loss="mean_squared_logarithmic_error", optimizer=Adadelta(learning_rate=i), metrics=["mean_squared_logarithmic_error"])
    model = NN_model.fit(x_train, y_train, epochs=250, batch_size=32, validation_split = 0.2,verbose=0)
    
    y_predict = NN_model.predict(x_test)
    r2 = r2_score(y_test,y_predict)
    RMSE = mean_squared_error(y_test,y_predict,squared=False)
    MSE = mean_squared_error(y_test,y_predict)
    
    results_Adadel[0,idx] = r2
    results_Adadel[1,idx] = RMSE
    results_Adadel[2,idx] = MSE
    results_Adadel[3,idx] = model
print('Execution time = ' + str(time.time() - start_time))

Execution time = 259.045512676239


In [17]:
print(results_Adadel)

[[0.6403294656984926 0.6405418669156349 0.6405747832287014
  0.6405807014804736 0.640581758607216]
 [905870.2959389403 905602.778378004 905561.3134836607 905553.8580219123
  905552.5263100262]
 [820600993064.5032 820116392205.9602 820041292478.2528 820027789778.3696
  820025377906.4707]


In [18]:
results_list = [results_Adam,results_Adadel,results_RMSprop,results_SGD]
results = [i[0,:] for i in results_list]
print(np.max(results))

0.6572490499980612


In [19]:
results

[array([0.6572490499980612, 0.6527524400278993, 0.6508330516621172,
        0.6506825074927765, 0.6506297371944765], dtype=object),
 array([0.6403294656984926, 0.6405418669156349, 0.6405747832287014,
        0.6405807014804736, 0.640581758607216], dtype=object),
 array([-194231243.9250411, -194231243.9250411, -194231243.9250411,
        -194231243.9250411, -194231243.9250411], dtype=object),
 array([0.6529187511298171, 0.6522290842930181, 0.6519354482977058,
        0.6519215143574366, 0.6519335561233899], dtype=object)]